## Concatenate Image Numpy Array
To make this simple, we are not going to scale the third dimension. Instead we are just going to take each slice of each image and assume it is a different example. This way we can concatenate all of the slices and we will end up with a giant matrix with dimenions equal to the output size for the first 2 dimensions (320x256) and then equal to the sum of the number of slices for all of the images for the third dimension. 

This actually signicantly increases our sample size. It goes from 80 images to 80x100 slices or so and gives us ~8000 images/examples to play with. 

One of the biggest limitaitons here is that there wont be context in that third dimension. For that reason, it might be interesting to add a "feature" to each image. That feature might be the normalized slice number so that we get an indication of whether it is on on the edge or in the middle of the image. This might drastically help the segmentation without requiring a 3D segmentation. Though, the 3D segmentation would be the ideal end result. 

In the next steps, it will be important to augment this dataset by making "more" examples. I will create more examples by flipping the images, and by creating small pertubrations that slighlty warp the images. Though, this will only be implemented once the basic functionality of the network is known to be working. 

In [1]:
import SimpleITK as sitk
import numpy
import os
import glob

In [23]:
paddedImagesLocation = '/Users/gattia/Data/mri/ski10Dataset/paddedImages/'
numpySaveLocation = '/Users/gattia/Data/mri/ski10Dataset/numpyTraining/'
trainImagesName = 'padded_training_images'
labelImagesName = 'padded_training_labels'
outputSize = [320,256]

In [21]:
def importImageAdd2Array(numpyArray, newImageName):
    image = sitk.ReadImage(newImageName)
    imageArray = sitk.GetArrayFromImage(image)
    concatenatedArray = numpy.concatenate((numpyArray, imageArray), axis=0)
    return(concatenatedArray)


In [22]:
concatenatedImages = numpy.empty([1,outputSize[0], outputSize[1]])
concatenatedLabels = numpy.empty([1,outputSize[0], outputSize[1]])

os.chdir(paddedImagesLocation)
imageNames = glob.glob('image-*-padded.mha')
labelNames = glob.glob('labels-*-padded.mha')
for imageName in imageNames:
    concatenatedImages = importImageAdd2Array(concatenatedImages, imageName)

for labelName in labelNames:
    concatenatedLabels = importImageAdd2Array(concatenatedLabels, labelName)

In [24]:
numpy.save(numpySaveLocation+trainImagesName+'.npy', concatenatedImages)
numpy.save(numpySaveLocation+labelImagesName+'.npy', concatenatedLabels)